# LangGraph on Vertex AI

- LangGraph エージェントを Vertex AI 上で実行するためのサンプルコード
- https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview?hl=ja


## デフォルトの LangGraphAgent で ReAct エージェントを作る


In [59]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory
from vertexai.preview.reasoning_engines import LanggraphAgent

model = "gemini-2.0-flash"

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

model_kwargs = {
    # temperature (float): The sampling temperature controls the degree of
    # randomness in token selection.
    "temperature": 0.28,
    # max_output_tokens (int): The token limit determines the maximum amount of
    # text output from one prompt.
    "max_output_tokens": 1000,
    # top_p (float): Tokens are selected from most probable to least until
    # the sum of their probabilities equals the top-p value.
    "top_p": 0.95,
    # top_k (int): The next token is selected from among the top-k most
    # probable tokens. This is not supported by all model versions. See
    # https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/image-understanding#valid_parameter_values
    # for details.
    "top_k": None,
    # safety_settings (Dict[HarmCategory, HarmBlockThreshold]): The safety
    # settings to use for generating content.
    # (you must create your safety settings using the previous step first).
    "safety_settings": safety_settings,
}


agent = LanggraphAgent(
    model=model,  # Required.
    model_kwargs=model_kwargs,  # Optional.
)

In [39]:
response = agent.query(
    input={
        "messages": [
            ("user", "USドルから日本円への為替レートを教えてください"),
        ]
    }
)

In [40]:
response

{'messages': [{'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'kwargs': {'content': 'USドルから日本円への為替レートを教えてください',
    'type': 'human',
    'id': 'bd328bbb-530f-4268-8d68-f8734ba0d606'}},
  {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'AIMessage'],
   'kwargs': {'content': '現在の為替レートは常に変動しています。リアルタイムの為替レートを知るためには、以下のいずれかの方法をお勧めします。\n\n*   **Googleなどの検索エンジンで「USD JPY」と検索する:** ほぼリアルタイムの為替レートが表示されます。\n*   **信頼できる金融情報サイトやアプリを利用する:** Bloomberg、Reuters、Yahoo! Financeなどが信頼できます。\n*   **銀行や両替所のウェブサイトを確認する:** 実際に取引を行う場合は、利用する金融機関の為替レートを確認してください。\n\n為替レートは常に変動するため、上記の方法で最新の情報を確認してください。\n',
    'response_metadata': {'is_blocked': False,
     'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH',
       'probability_label': 'NEGLIGIBLE',
       'probability_score': 6.99526708558551e-07,
       'blocked': False,
       'severity': 'HARM_SEVERITY_NEGLIGIBLE',
       'severity_score': 0.007363021373748779},
     

In [41]:
len(response["messages"])

2

In [42]:
response["messages"][0]["kwargs"]["content"]

'USドルから日本円への為替レートを教えてください'

In [43]:
response["messages"][1]["kwargs"]["content"]

'現在の為替レートは常に変動しています。リアルタイムの為替レートを知るためには、以下のいずれかの方法をお勧めします。\n\n*   **Googleなどの検索エンジンで「USD JPY」と検索する:** ほぼリアルタイムの為替レートが表示されます。\n*   **信頼できる金融情報サイトやアプリを利用する:** Bloomberg、Reuters、Yahoo! Financeなどが信頼できます。\n*   **銀行や両替所のウェブサイトを確認する:** 実際に取引を行う場合は、利用する金融機関の為替レートを確認してください。\n\n為替レートは常に変動するため、上記の方法で最新の情報を確認してください。\n'

## 独自ツールを与える


In [44]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain
    exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code).
            Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code).
            Defaults to "EUR" (Euro).
        currency_date: The date for which to retrieve the exchange rate.
            Defaults to "latest" for the most recent exchange rate data.
            Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24",
                "rates": {"EUR": 0.95534}}
    """
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [6]:
# ツールが動作するか確認
get_exchange_rate(currency_from="USD", currency_to="JPY")

{'amount': 1.0, 'base': 'USD', 'date': '2025-05-08', 'rates': {'JPY': 144.68}}

In [48]:
from vertexai.preview.reasoning_engines import LanggraphAgent

agent = LanggraphAgent(
    model=model,
    tools=[get_exchange_rate],
)

In [49]:
response = agent.query(
    input={
        "messages": [
            ("user", "USドルから日本円への為替レートを教えてください"),
        ]
    }
)

In [50]:
for msg in response["messages"]:
    print(msg["kwargs"]["content"])

USドルから日本円への為替レートを教えてください

{"amount": 1.0, "base": "USD", "date": "2025-05-08", "rates": {"JPY": 144.68}}
 現在の米ドルから日本円への為替レートは、1米ドルあたり144.68円です。


## 任意のアーキテクチャの LangGraphAgent を作る


In [ ]:
def langgraph_builder(*, model, **kwargs):
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser
    from langgraph.graph import END, MessageGraph

    output_parser = StrOutputParser()

    # LanggraphAgentにあたえた文字列のmodelのオブジェクトが自動的に作られている
    print("*******", type(model))

    planner = (
        ChatPromptTemplate.from_template("Generate an argument about: {input}")
        | model
        | output_parser
    )

    pros = (
        ChatPromptTemplate.from_template("List the positive aspects of {input}")
        | model
        | output_parser
    )

    cons = (
        ChatPromptTemplate.from_template("List the negative aspects of {input}")
        | model
        | output_parser
    )

    summary = (
        ChatPromptTemplate.from_template(
            "Input:{input}\nGenerate a final response given the critique",
        )
        | model
        | output_parser
    )

    builder = MessageGraph()
    builder.add_node("planner", planner)
    builder.add_node("pros", pros)
    builder.add_node("cons", cons)
    builder.add_node("summary", summary)

    builder.add_edge("planner", "pros")
    builder.add_edge("planner", "cons")
    builder.add_edge("pros", "summary")
    builder.add_edge("cons", "summary")
    builder.add_edge("summary", END)
    builder.set_entry_point("planner")
    return builder.compile()

In [70]:
agent = LanggraphAgent(model=model, runnable_builder=langgraph_builder)
agent

In [ ]:
response = agent.query(input={"role": "user", "content": "スクラムの利点と欠点"})
response

******* <class 'langchain_google_vertexai.chat_models.ChatVertexAI'>


[{'lc': 1,
  'type': 'constructor',
  'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
  'kwargs': {'content': 'スクラムの利点と欠点',
   'type': 'human',
   'id': 'be8b5df5-7247-4d32-9a9f-3f3362220547'}},
 {'lc': 1,
  'type': 'constructor',
  'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
  'kwargs': {'content': '## スクラムの利点と欠点：銀の弾丸ではない、状況適合が重要\n\nスクラムは、複雑なプロジェクトを管理するための人気のあるアジャイルフレームワークですが、万能薬ではありません。確かに多くの利点がありますが、同時に克服すべき欠点も存在します。スクラムの導入を検討する際には、これらの両側面を理解し、プロジェクトやチームの特性に合わせた適切な適用を心がける必要があります。\n\n**スクラムの利点:**\n\n*   **柔軟性と適応性:** スクラムは、変化する要件や市場のニーズに迅速に対応できます。短いスプリントサイクルを通して、定期的に進捗状況をレビューし、必要に応じて方向修正を行うことができます。これにより、開発チームは柔軟に対応し、無駄な開発を防ぐことができます。\n*   **透明性の向上:** スクラムは、デイリースクラム、スプリントレビュー、スプリントレトロスペクティブといったイベントを通じて、プロジェクトの進捗状況、課題、改善点を可視化します。ステークホルダーも参加することで、チーム全体が共通の認識を持ち、スムーズなコミュニケーションを実現できます。\n*   **チームのエンパワーメント:** スクラムは、自己組織化されたチームを重視します。プロダクトオーナーが何を開発するかを定義し、開発チームがどのように開発するかを決定します。これにより、チームメンバーは責任感を持ち、主体的にプロジェクトに取り組むことができます。\n*   **品質の向上:** スプリントごとに動くソフトウェアを開発し、レビューを行うこ

## エージェントデプロイ


In [ ]:
import vertexai
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory
from vertexai.preview.reasoning_engines import LanggraphAgent, ReasoningEngine

PROJECT_ID = "inbound-mote-433711-a6"
LOCATION = "us-central1"

model = "gemini-2.0-flash"

vertexai.init(
    project=PROJECT_ID, location=LOCATION, staging_bucket="gs://agent-demo-20250509"
)

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

model_kwargs = {
    # temperature (float): The sampling temperature controls the degree of
    # randomness in token selection.
    "temperature": 0.28,
    # max_output_tokens (int): The token limit determines the maximum amount of
    # text output from one prompt.
    "max_output_tokens": 1000,
    # top_p (float): Tokens are selected from most probable to least until
    # the sum of their probabilities equals the top-p value.
    "top_p": 0.95,
    # top_k (int): The next token is selected from among the top-k most
    # probable tokens. This is not supported by all model versions. See
    # https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/image-understanding#valid_parameter_values
    # for details.
    "top_k": None,
    # safety_settings (Dict[HarmCategory, HarmBlockThreshold]): The safety
    # settings to use for generating content.
    # (you must create your safety settings using the previous step first).
    "safety_settings": safety_settings,
}


def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain
    exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code).
            Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code).
            Defaults to "EUR" (Euro).
        currency_date: The date for which to retrieve the exchange rate.
            Defaults to "latest" for the most recent exchange rate data.
            Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24",
                "rates": {"EUR": 0.95534}}
    """
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()


agent = LanggraphAgent(
    model=model,
    tools=[get_exchange_rate],
    model_kwargs=model_kwargs,
)

In [3]:
# ローカルで試す
response = agent.query(
    input={
        "messages": [
            ("user", "USドルから日本円への為替レートを教えてください"),
        ]
    }
)

print(response["messages"][-1]["kwargs"]["content"])

 現在の米ドルから日本円への為替レートは、1米ドルあたり144.68円です。


In [4]:
# デプロイ
remote_agent = ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "cloudpickle>=3.0.0",
        "langgraph>=0.2.76",
        "pydantic>=2.10",
    ],
    display_name="Exchange Rate Agent",
    description="An agent that retrieves exchange rates between currencies.",
    extra_packages=[],
)

Using bucket agent-demo-20250509
Writing to gs://agent-demo-20250509/reasoning_engine/reasoning_engine.pkl
Writing to gs://agent-demo-20250509/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://agent-demo-20250509/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672/operations/5572900626985123840
ReasoningEngine created. Resource name: projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672')


In [5]:
remote_agent

resource name: projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672

## デプロイしたエージェントを使用する


In [7]:
remote_agent.list()

 resource name: projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672]

In [8]:
remote_agent.operation_schemas()

[{'parameters': {'type': 'object',
   'properties': {'input': {'anyOf': [{'type': 'string'},
      {'additionalProperties': True, 'type': 'object'}]},
    'config': {'nullable': True}},
   '$defs': {'RunnableConfig': {'title': 'RunnableConfig',
     'type': 'object',
     'properties': {'configurable': {'title': 'Configurable',
       'type': 'object',
       'additionalProperties': True},
      'max_concurrency': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
       'title': 'Max Concurrency'},
      'tags': {'title': 'Tags', 'type': 'array', 'items': {'type': 'string'}},
      'metadata': {'title': 'Metadata',
       'type': 'object',
       'additionalProperties': True},
      'recursion_limit': {'title': 'Recursion Limit', 'type': 'integer'},
      'run_name': {'title': 'Run Name', 'type': 'string'},
      'run_id': {'anyOf': [{'format': 'uuid', 'type': 'string'},
        {'type': 'null'}],
       'title': 'Run Id'},
      'callbacks': {'anyOf': [{'type': 'array', 'items': {}},

In [9]:
remote_agent = ReasoningEngine(
    "projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672"
)
response = remote_agent.query(
    input={
        "messages": [
            ("user", "USドルから日本円への為替レートを教えてください"),
        ]
    }
)

In [11]:
response["messages"][-1]["kwargs"]["content"]

' 現在の米ドルから日本円への為替レートは、1米ドルあたり144.68円です。'

## ストリーム


In [16]:
remote_agent = ReasoningEngine(
    "projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672"
)

stream_response = remote_agent.stream_query(
    input={
        "messages": [
            ("user", "USドルから日本円への為替レートを教えてください"),
        ]
    }
)

In [17]:
for msg in stream_response:
    print(msg)
    print("=" * 100)

{'agent': {'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_exchange_rate', 'arguments': '{"currency_to": "JPY", "currency_from": "USD"}'}}, 'response_metadata': {'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 7.462618327735981e-07, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.0}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 1.7845194406618248e-06, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.01813860982656479}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 1.431556142961199e-06, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.0}, {'category': 'HARM_CATEGORY_SEXUAL

In [ ]:
remote_agent = ReasoningEngine(
    "projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672"
)

# langgraphのstreamと同じようにupdatesとmessagesモードが使える
stream_response = remote_agent.stream_query(
    input={
        "messages": [
            ("user", "USドルから日本円への為替レートを教えてください"),
        ]
    },
    stream_mode=["updates", "messages"],
)

In [19]:
for msg in stream_response:
    print(msg)
    print("=" * 100)

['messages', [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_exchange_rate', 'arguments': '{"currency_to": "JPY", "currency_from": "USD"}'}}, 'response_metadata': {'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 7.462618327735981e-07, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.0}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 1.7845194406618248e-06, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.01813860982656479}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 1.431556142961199e-06, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.0}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability

## エージェントを削除する


In [20]:
remote_agent.delete()

Deleting ReasoningEngine : projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672
ReasoningEngine deleted. . Resource name: projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672
Deleting ReasoningEngine resource: projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672
Delete ReasoningEngine backing LRO: projects/746152150924/locations/us-central1/operations/7050081304762646528
ReasoningEngine resource projects/746152150924/locations/us-central1/reasoningEngines/6523261750106652672 deleted.
